# Introduction

A family of Apicomplexan Apetala2 (ApiAP2) proteins has been identified in malaria parasites. However, while some members are transcriptional regulators the function of other members remains unclear. As part of a larger study, [Modrzynska _et al._](https://doi.org/10.1016/j.chom.2016.12.003) systematically knocked out ApiAP2 proteins in the rodent malaria parasite **_Plasmodium berghei_** to try and identify their function.

One member of the ApiAP2 family, **ApiAP2-o**, plays a key role in ookinete development. Modrzynska _et al._ generated a series of ApiAP2 knockout lines as part of a larger study to elucidate the function of these genes.

**A Knockout Screen of ApiAP2 Genes Reveals Networks of Interacting Transcriptional Regulators Controlling the _Plasmodium_ Life Cycle**  
Modrzynska K., Pfander C., Chappell L., Yu L., Suarez C., Dundas K., Gomes A.R., Goulding D., Rayner J.C., Choudhary J., Billker O.  
_Cell Host Microbe. 2017;21(1):11-22 doi:[10.1016/j.chom.2016.12.003](https://doi.org/10.1016/j.chom.2016.12.003)_

# Dataset

The group task uses a subset of the Modrzynska _et al._ dataset to compare the transcriptomes from ApiAP2-o knockouts (**KO**) with those from wild type (**WT**) parasites. Below is a table giving an overview of the samples used in the group task.

| Sample name | Experimental Condition | Replicate | GEO accession | ENA run |
| :-: | :-: | :-: | :-: | :-: |
| WT1 | wild type | 1 | [GSM2131969](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131969) | [SRR3437923](https://www.ebi.ac.uk/ena/data/view/SRR3437923) |
| WT2 | wild type | 2 | [GSM2131976](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131976) | [SRR3437938](https://www.ebi.ac.uk/ena/data/view/SRR3437938) |
| WT3 | wild type | 3 | [GSM2131984](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131984) | [SRR3437953](https://www.ebi.ac.uk/ena/data/view/SRR3437953) |
| KO1 | AP2-o knockout | 1 | [GSM2131972](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131972) | [SRR3437929](https://www.ebi.ac.uk/ena/data/view/SRR3437929) |
| KO2 | AP2-o knockout | 2 | [GSM2131980](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131980) | [SRR3437946](https://www.ebi.ac.uk/ena/data/view/SRR3437946) |
| KO3 | AP2-o knockout | 3 | [GSM2131988](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2131988) | [SRR3437962](https://www.ebi.ac.uk/ena/data/view/SRR3437962) |

## Downloading the raw FASTQ files

The script `download_raw_data.sh` uses the information in `samples.txt` and `curl` to download the raw FASTQ files from the [ENA](https://www.ebi.ac.uk/ena) and rename them. The raw data files are named using the format  `[sample]_raw.fq.gz`. Be patient, the script may take a while to run.

In [ ]:
chmod +x download_raw_data.sh
./download_raw_data.sh

These raw data files are probably too large for a group task to process in the time allowed. Probably need to downsample with something like Picard. The example below will yield ~2-3 million reads per sample.

_N.B. can't just skim first 2 million reads as the FASTQ files are alignment sorted._

In [ ]:
echo "Indexing genome..."
hisat2-build PbANKA_v3.fasta PbANKA_v3_hisat2.idx

for fname in *_1_raw.fq.gz
do
    # Get sample name from file name
    sample=`echo "$fname" | cut -d'_' -f1`

    # Align sample to genome
    echo "Aligning sample..."${sample}
    hisat2 --max-intronlen 10000 -x PbANKA_v3_hisat2.idx \
    -1 ${sample}_1.fq.gz -2 ${sample}_2.fq.gz \
    -S ${sample}.sam -p 4
    
    # Downsample SAM file using Picard
    echo "Downsampling alignment..."${sample}
    java -jar DownsampleSam.jar I=${sample}.sam O=${sample}.downsampled.sam P=0.4

    # Extract reads from downsampled SAM using Picard
    echo "Extracting reads from downsampled alignment..."${sample}
    java -jar SamToFastq.jar I=${sample}.downsampled.sam F=${sample}_1.fq F2=${sample}_2.fq
    
    # Compress FASTQ files
    echo "Compressing FASTQ files..."${sample}
    gzip ${sample}_1.fq
    gzip ${sample}_2.fq
done

You can run the following command to remove the raw data files.

In [ ]:
rm ./*_raw.fq.gz
rm ./*.sam